In [5]:
import pandas as pd 
data = pd.read_csv('https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/ff414a1bcfcba32481e4d4e8db578e55872a2ca1/titanic.csv', sep='\t')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
import numpy as np
data = data[np.isfinite(data['Age'])]

In [23]:
features_list = ['Age', 'Fare', 'Pclass', 'SibSp']
X, y = data[features_list], data['Survived']

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

clf = KNeighborsClassifier() # инициализируем классификатор с начальными параметрами. Сейчас классификатор не обучен
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8) # разобьём выборку на обучающую и на тестовую (80% данных на обучение)
clf.fit(X_train, y_train) # обучим классификатор, указав ему, какие объекты (X) какие метки классов (y) имеют. Иными словами, какое значение y соответствует каждому объекту из X
y_pred = clf.predict(X_test) # получим предсказания обученного классификатора для тестовой выборки

C:\Users\Tom\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6538461538461539

In [26]:
def square_distance(x):
    return 1/x*2

params_grid = {
    'n_neighbors': range(1,100,2), # Перебираем от 1 до 100 с шагом 2
    'weights': ['uniform', 'distance', square_distance], # с каким весом берём каждого соседа. uniform - с одинаковым, distance - обратно пропорционально расстоянию. Ещё можно передать свою функцию
    'algorithm': ['auto'], # алгоритм поиска ближайших соседей. Влияет только на скорость обучения
    'leaf_size': [30], # параметры некоторых алгоритмов поиска ближ соседей. Влияет только на скорость обучения
    'metric': ['minkowski'], # метрика расстояния, используемая в алгоритме поиска ближайших соседей
    'p': [2] # параметр метрики, используемой в методе ближайших соседей 
}

In [27]:
params_dicts = [{}]

for param_name, param_values in params_grid.items():
    new_params_dicts = []
    for old_param_dict in params_dicts:
        for param_value in param_values:
            new_param_dict = old_param_dict.copy()
            new_param_dict[param_name] = param_value
            new_params_dicts.append(new_param_dict)
    params_dicts = new_params_dicts

from operator import mul
from functools import reduce
    
assert len(params_dicts) == reduce(mul, [len(param_values) for param_values in params_grid.values()], 1)

best_acc = 0
best_params = {}

for params in params_dicts:
    clf = KNeighborsClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_params = params

print('Best accuracy: %f' % best_acc)
print(best_params)

C:\Users\Tom\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


Best accuracy: 0.730769
{'n_neighbors': 7, 'weights': 'distance', 'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'p': 2}
